In [1]:
# choose environment = crosscoders_p312

from utils import *
import torch
from trainer import Trainer

device = 'cuda:0'
# torch.set_grad_enabled(False) # important for memory
base_model_id = "Qwen/Qwen2.5-0.5B" # "Qwen/Qwen2.5-1.5B",
chat_model_id = "Qwen/Qwen2.5-0.5B-Instruct"  # "Qwen/Qwen2.5-1.5B-Instruct", 
# local_dataset_path = "data/pile-lmsys-mix-300k-tokenized-qwen2x5"
# hf_dataset_path = "aastha6/pile-lmsys-mix-300k-tokenized-qwen2.5"
local_dataset_path = "data/pile-lmsys-mix-500k-tokenized-qwen2x5"
hf_dataset_path = "aastha6/pile-lmsys-mix-500k-tokenized-qwen2.5"


base_model = HookedTransformer.from_pretrained(
    base_model_id,
    device=device, 
    dtype=torch.bfloat16
)

chat_model = HookedTransformer.from_pretrained(
    chat_model_id,
    device=device,
    dtype=torch.bfloat16 
)

all_tokens = load_pile_lmsys_mixed_tokens(
    local_dataset_path=local_dataset_path,
    hf_dataset_path=hf_dataset_path,
)

n_layers = base_model.cfg.n_layers
d_model = base_model.cfg.d_model
n_heads = base_model.cfg.n_heads
d_head = base_model.cfg.d_head
d_mlp = base_model.cfg.d_mlp
d_vocab = base_model.cfg.d_vocab

print(f"n_layers: {n_layers}")
print(f"d_model: {d_model}")
print(f"n_heads: {n_heads}")
print(f"d_head: {d_head}")
print(f"d_mlp: {d_mlp}")
print(f"d_vocab: {d_vocab}")

model_name = base_model_id.split("/")[-1].lower()
print(f"model_name: {model_name}")

/home/ubuntu/miniconda3/envs/crosscoders_p312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loaded pretrained model Qwen/Qwen2.5-0.5B into HookedTransformer


Loaded pretrained model Qwen/Qwen2.5-0.5B-Instruct into HookedTransformer
Data is not cached. Loading data from HF


Saving the dataset (9/9 shards): 100%|██████████| 536465/536465 [00:44<00:00, 12095.47 examples/s]


Saved tokens to disk
n_layers: 24
d_model: 896
n_heads: 14
d_head: 64
d_mlp: 4864
d_vocab: 151936
model_name: qwen2.5-0.5b


In [2]:
default_cfg = {
    "seed": 49,
    "batch_size": 4096, # 128
    "buffer_mult": 128,
    "lr": 5e-5,
    # "num_tokens": 200_000_000,
    "num_tokens": 500_000_000,
    "l1_coeff": 2,
    "beta1": 0.9,
    "beta2": 0.999,
    "d_in": base_model.cfg.d_model,
    "dict_size": 2**14,
    "seq_len": 1024,
    "enc_dtype": "fp32",
    "model_name": model_name,
    "site": "resid_pre",
    "device": "cuda:0",
    "model_batch_size": 4, # 1
    "log_every": 100,
    "save_every": 30000,
    "dec_init_norm": 0.08,
    "hook_point": "blocks.12.hook_resid_pre",
    "wandb_project": "crosscoders-r1-distill-qwen",
    "wandb_entity": "varmaaastha6",
}
cfg = arg_parse_update_cfg(default_cfg)

trainer = Trainer(cfg, base_model, chat_model, all_tokens)

In IPython - skipped argparse


100%|██████████| 128/128 [00:30<00:00,  4.14it/s]
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: varmaaastha6 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
%%time
trainer.train()

  4%|▍         | 4639/122070 [11:57<2:27:30, 13.27it/s] 

In [ ]:
# 122070

In [ ]:
# Run data is saved locally in /home/ubuntu/crosscoders-r1-distill-qwen/wandb/run-20250227_153033-zihcmacm
# Syncing run rich-serenity-23 to Weights & Biases (docs)
# View project at https://wandb.ai/varmaaastha6/crosscoders-r1-distill-qwen
# View run at https://wandb.ai/varmaaastha6/crosscoders-r1-distill-qwen/runs/zihcmacm


# Saved as version 1 in /home/ubuntu/crosscoders-r1-distill-qwen/checkpoints/version_2
# final: saved model
# CPU times: user 2h 9min 29s, sys: 15.4 s, total: 2h 9min 44s
# Wall time: 2h 6min 53s

In [ ]:
import os
from pathlib import Path
import huggingface_hub

def push_to_hub(local_dir):
    if isinstance(local_dir, huggingface_hub.Repository):
        local_dir = local_dir.local_dir
    os.system(f"git -C {local_dir} add .")
    os.system(f"git -C {local_dir} commit -m 'Auto Commit'")
    os.system(f"git -C {local_dir} push origin main --force")


def upload_folder_to_hf(folder_path, repo_name=None, debug=False):
    """
    Uploads a folder to HuggingFace, and creates a repo for it.
    """
    folder_path = Path(folder_path)
    if repo_name is None:
        repo_name = folder_path.name
    repo_folder = folder_path.parent / (folder_path.name + "_repo")
    print(f"repo_folder: {repo_folder}")
    print(f"folder_path: {folder_path}")
    repo_url = huggingface_hub.create_repo(repo_name, exist_ok=True)
    repo = huggingface_hub.Repository(repo_folder, repo_url)

    for file in folder_path.iterdir():
        if debug:
            print(file.name)
        file.rename(repo_folder / file.name)
    push_to_hub(repo.local_dir)
   
model_name = "qwen2.5-0.5b"
upload_folder_to_hf(folder_path=f"checkpoints/version_2", repo_name=f"crosscoders-{model_name}", debug=True)

In [ ]:
# 300k dataste

# /home/ubuntu/miniconda3/envs/crosscoders_p312/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
# For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
#   warnings.warn(warning_message, FutureWarning)
# Cloning https://huggingface.co/aastha6/crosscoders-qwen2.5-0.5b into local empty directory.
# WARNING:huggingface_hub.repository:Cloning https://huggingface.co/aastha6/crosscoders-qwen2.5-0.5b into local empty directory.
# repo_folder: checkpoints/version_2_repo
# folder_path: checkpoints/version_2
# Moving checkpoints/version_2/0.pt to checkpoints/version_2_repo/0.pt
# Moving checkpoints/version_2/0_cfg.json to checkpoints/version_2_repo/0_cfg.json
# Moving checkpoints/version_2/1.pt to checkpoints/version_2_repo/1.pt
# Moving checkpoints/version_2/1_cfg.json to checkpoints/version_2_repo/1_cfg.json
# [main 4f81f30] Auto Commit
#  4 files changed, 8 insertions(+)
#  create mode 100644 0.pt
#  create mode 100644 0_cfg.json
#  create mode 100644 1.pt
#  create mode 100644 1_cfg.json
# Uploading LFS objects: 100% (2/2), 480 MB | 56 MB/s, done.
# To https://huggingface.co/aastha6/crosscoders-qwen2.5-0.5b
#    593aea3..4f81f30  main -> main